To load the entire data, it is recommended to request the following resources on SLURM:

`srun -p himem -c 8 --mem 64GB -t 0-08:00:00 --pty bash`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

from pyspark.sql import SparkSession

from make_clinical_dataset.constants import ROOT_DIR

In [ ]:
date = '2025-01-08'
data_dir = f'{ROOT_DIR}/data/raw/data_pull_{date}/observation_parquet'

In [ ]:
!du -hs {root_dir}/data/raw/data_pull_{date}/observation_csv
!du -hs {root_dir}/data/raw/data_pull_{date}/observation_parquet

In [ ]:
# Initialize a Spark session
spark = SparkSession.builder.appName("EPIC-data").config("spark.driver.memory", "50G").getOrCreate()

# Read all Parquet files in the folder
# make sure its all parquet.gzip files in this folder (no one created some weird files in there)
assert all([fname.endswith('parquet.gzip') for fname in os.listdir(data_dir)])
df = spark.read.parquet(data_dir)

In [ ]:
# Print schema
df.printSchema()

In [ ]:
# Total data size
print((df.count(), len(df.columns)))

In [ ]:
# Total number of unique patients
df.select("PATIENT_RESEARCH_ID").distinct().count()

In [ ]:
# Total number of unique procedure names
df.select('proc_name').distinct().count()

In [ ]:
# BONUS: Spark's data partition distribution
from pyspark.sql.functions import spark_partition_id
df.groupBy(spark_partition_id()).count().orderBy("count", ascending=False).show(n=df.rdd.getNumPartitions())